In [ ]:
import aoc
import re

In [ ]:
def pairs_in_template(polymer_template):
    for idx in range(len(polymer_template) - 1):
        print(polymer_template[idx:idx + 2])

In [ ]:
pairs_in_template('NNCB')

In [ ]:
example_insertion_rules = r'''CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C'''

In [ ]:
def insertion_rules_dict(insertion_rules_string):
    insertion_rules = re.findall(r'(\w\w) -> (\w)', insertion_rules_string)
    return {r[0]: r[1] for r in insertion_rules}

In [ ]:
example_rules = insertion_rules_dict(example_insertion_rules)

In [ ]:
def apply_rules_to_template(polymer_template, insertion_rules):
    polymer_entries = []
    for idx in range(len(polymer_template) - 1):
        polymer_entries.append((idx, polymer_template[idx]))
        pair = polymer_template[idx:idx + 2]
        if pair in insertion_rules:
            polymer_entries.append((idx + 0.5, insertion_rules[pair]))
    polymer_entries.append((idx + 1, polymer_template[-1]))
    polymer_entries.sort()
    return ''.join([p[1] for p in polymer_entries])

In [ ]:
apply_rules_to_template('NNCB', example_rules)

In [ ]:
apply_rules_to_template('NCNBCHB', example_rules)

In [ ]:
def polymerise(polymer_template, insertion_rules, num_steps):
    for k in range(num_steps):
        polymer_template = apply_rules_to_template(polymer_template, insertion_rules)
    return polymer_template

In [ ]:
polymerise('NNCB', example_rules, 2)

In [ ]:
polymer10 = polymerise('NNCB', example_rules, 10)

In [ ]:
assert len(polymer10) == 3073

In [ ]:
element_counts = aoc.distinct_counts(polymer10)
most_common_element = aoc.most_common_entry(polymer10)
least_common_element = aoc.least_common_entry(polymer10)

In [ ]:
assert element_counts[most_common_element] == 1749
assert element_counts[least_common_element] == 161

In [ ]:
day14_string = aoc.read_file_as_string('inputs/day14.txt')

In [ ]:
day14_rules = insertion_rules_dict(day14_string)

In [ ]:
len(day14_rules)

In [ ]:
day14_polymer_template = aoc.read_file_as_list('inputs/day14.txt')[0]

In [ ]:
day14_polymer_template

In [ ]:
polymer10 = polymerise(day14_polymer_template, day14_rules, 10)

In [ ]:
element_counts = aoc.distinct_counts(polymer10)
most_common_element = aoc.most_common_entry(polymer10)
least_common_element = aoc.least_common_entry(polymer10)

In [ ]:
element_counts[most_common_element] - element_counts[least_common_element]

In [ ]:
# star 2

In [ ]:
polymer10 = polymerise('NNCB', example_rules, 10)

In [ ]:
polymer10a = polymerise('NN', example_rules, 10)
polymer10b = polymerise('NC', example_rules, 10)
polymer10c = polymerise('CB', example_rules, 10)

In [ ]:
alt_route = polymer10a + polymer10b[1:-1] + polymer10c

In [ ]:
len(polymer10)

In [ ]:
len(alt_route)

In [ ]:
alt_route == polymer10

In [ ]:
%time x=polymerise('FS', day14_rules, 24)

In [ ]:
2**16

In [ ]:
def direct_count_for_pair(pair, insertion_rules, num_steps):
    polymer = polymerise(pair, insertion_rules, num_steps)
    return aoc.distinct_counts(polymer)

In [ ]:
direct_count_for_pair('FS', day14_rules, 1)

In [ ]:
def add_dicts(x, y):
    return {k: x.get(k, 0) + y.get(k, 0) for k in set(x) | set(y)}

In [ ]:
counts1 = aoc.distinct_counts(polymerise('FSK', day14_rules, 1))
counts1

In [ ]:
direct_count_for_pair('FS', day14_rules, 1)

In [ ]:
direct_count_for_pair('SK', day14_rules, 1)

In [ ]:
add_dicts(direct_count_for_pair('FS', day14_rules, 1), direct_count_for_pair('SK', day14_rules, 1))

In [ ]:
def recurse_counts_for_pair(pair, insertion_rules, num_steps):
    if num_steps <= 10:
        polymer = polymerise(pair, insertion_rules, num_steps)
        return aoc.distinct_counts(polymer)
    elif pair not in insertion_rules:
        return {pair[0]: 1, pair[1]:1}
    else:
        insert = insertion_rules[pair]
        counts_left = recurse_counts_for_pair(pair[0]+insert, insertion_rules, num_steps - 1)
        counts_right = recurse_counts_for_pair(insert+pair[1], insertion_rules, num_steps - 1)
        counts = add_dicts(counts_left, counts_right)
        counts[insert] -= 1
        return counts

In [ ]:
%time recurse_counts_for_pair('FS', day14_rules, 10)

In [ ]:
%time recurse_counts_for_pair('FS', day14_rules, 11)

In [ ]:
%time recurse_counts_for_pair('FS', day14_rules, 12)

In [ ]:
%time recurse_counts_for_pair('FS', day14_rules, 25)

In [ ]:
%time aoc.distinct_counts(polymerise('FS', day14_rules, 25))

In [ ]:
class Polymerizer():
    def __init__(self, insertion_rules_string):
        insertion_rules = re.findall(r'(\w\w) -> (\w)', insertion_rules_string)
        self.insertion_rules = {r[0]: r[1] for r in insertion_rules}
    
    def apply_rules_to_template(self, polymer_template):
        polymer_entries = []
        for idx in range(len(polymer_template) - 1):
            polymer_entries.append((idx, polymer_template[idx]))
            pair = polymer_template[idx:idx + 2]
            if pair in self.insertion_rules:
                polymer_entries.append((idx + 0.5, self.insertion_rules[pair]))
        polymer_entries.append((idx + 1, polymer_template[-1]))
        polymer_entries.sort()
        return ''.join([p[1] for p in polymer_entries])
    
    def polymerize(self, polymer_template, num_steps):
        for k in range(num_steps):
            polymer_template = self.apply_rules_to_template(polymer_template)
        return polymer_template
    
    def counts_after_polymerization(self, polymer_template, num_steps):
        return aoc.distinct_counts(self.polymerize(polymer_template, num_steps))
    
    def score_after_polymerization(self, polymer_template, num_steps):
        element_counts = self.counts_after_polymerization(polymer_template, num_steps)
        ordered_counts = sorted([(v, k) for k, v in element_counts.items()])
        return ordered_counts[-1][0] - ordered_counts[0][0]
    

In [ ]:
example_polymerizer = Polymerizer(example_insertion_rules)

In [ ]:
example_polymerizer.score_after_polymerization('NNCB', 10)

In [ ]:
full_polymerizer = Polymerizer(day14_string)

In [ ]:
full_polymerizer.score_after_polymerization(day14_polymer_template, 10)

In [ ]:
class RecursivePolymerizer():
    def __init__(self, insertion_rules_string):
        insertion_rules = re.findall(r'(\w\w) -> (\w)', insertion_rules_string)
        self.insertion_rules = {r[0]: r[1] for r in insertion_rules}
        self.cache = {}
    
    def apply_rules_to_template(self, polymer_template):
        polymer_entries = []
        for idx in range(len(polymer_template) - 1):
            polymer_entries.append((idx, polymer_template[idx]))
            pair = polymer_template[idx:idx + 2]
            if pair in self.insertion_rules:
                polymer_entries.append((idx + 0.5, self.insertion_rules[pair]))
        polymer_entries.append((idx + 1, polymer_template[-1]))
        polymer_entries.sort()
        return ''.join([p[1] for p in polymer_entries])
    
    def polymerize(self, polymer_template, num_steps):
        for k in range(num_steps):
            polymer_template = self.apply_rules_to_template(polymer_template)
        return polymer_template
    
    def counts_after_polymerization_of_pair(self, pair, num_steps, cache=True):
        # Consult cache for perfomance boost
        if cache and (pair, num_steps) in self.cache:
            #print(f'retrieved {pair}, {num_steps}')
            #print(self.cache[(pair, num_steps)])
            return self.cache[(pair, num_steps)]
        else:
            if pair not in self.insertion_rules:
                print('Nothing to do for ', pair)
                counts = aoc.distinct_counts(pair)
            elif num_steps == 1:
                counts = aoc.distinct_counts([pair[0], pair[1], self.insertion_rules[pair]])
            else:
                insertion = self.insertion_rules[pair]
                counts_left = self.counts_after_polymerization_of_pair(pair[0] + insertion, num_steps - 1, cache)
                counts_right = self.counts_after_polymerization_of_pair(insertion + pair[1], num_steps - 1, cache)
                counts = add_dicts(counts_left, counts_right)
                counts[insertion] -= 1
                
            self.cache[(pair, num_steps)] = {k:v for k,v in counts.items()}
            #print(f'calculated {pair}, {num_steps}')
            #print(counts)
            return counts

    def counts_after_polymerization(self, polymer_template, num_steps, cache=True):
        scores = {}
        for idx in range(len(polymer_template) - 1):
            pair = polymer_template[idx: idx + 2]
            scores_idx = self.counts_after_polymerization_of_pair(pair, num_steps, cache)
            scores = add_dicts(scores, scores_idx)
            
        # Correct for repeating characters
        for char in polymer_template[1:-1]:
            print(f'removing a {char}')
            scores[char] -= 1
            
        return scores
    
    def score_after_polymerization(self, polymer_template, num_steps):
        element_counts = self.counts_after_polymerization(polymer_template, num_steps)
        ordered_counts = sorted([(v, k) for k, v in element_counts.items()])
        return ordered_counts[-1][0] - ordered_counts[0][0]
        
    

In [ ]:
%time example_polymerizer.counts_after_polymerization('NNCB', 20)

In [ ]:
example_polymerizer2 = RecursivePolymerizer(example_insertion_rules)


In [ ]:
%time example_polymerizer2.score_after_polymerization('NNCB', 40)

In [ ]:
full_polymerizer = Polymerizer(day14_string)

In [ ]:
full_polymerizer

In [ ]:
full_polymerizer2 = RecursivePolymerizer(day14_string)
assert full_polymerizer2.score_after_polymerization(day14_polymer_template, 10) == 2360

In [ ]:
%time full_polymerizer2.score_after_polymerization(day14_polymer_template, 40)